In [4]:
import av as av
import collections.abc
collections.Iterable = collections.abc.Iterable
import pims
import trackpy as tp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import fft
from sklearn.decomposition import PCA ## need this for principle component analysis below
from matplotlib.mlab import psd
from scipy.optimize import minimize, curve_fit
from scipy.stats import chi2
import os


#make a pipeline so that when pims opens a file, it converts each frame to one color
@pims.pipeline
def gray(image):
    return image[:, :, 1]  # Take just the green channel (they are all the same for our camera)


def processmovie(filename, framerate):
    #open a avi file with pims and converts to one color
    spheres = gray(pims.open(filename))
    tp.quiet()
    #process every frame in the tiff image stack to find the locations of bright spots
    #minmass defines the minimum brightness and processes means no parallelization since that breaks it
    #invert=true looks for dark spots instead of light spots
    f = tp.batch(spheres[:], 31, invert=False, minmass=1000, processes='auto')
    #to check the mass brightness make this figure
    fig, ax = plt.subplots()
    ax.hist(f['mass'], bins=100)

    return [spheres, f]
    


In [5]:

def PSDmaker(spheres, f):
    tp.quiet()
    #look at the location in each frame and labels them. It looks for maximum 5 pixel movement between frames
    #if it vanishes for one frame, memory prevents it from thinking the sphere is gone (up to 3 frames)
    t = tp.link(f, 20, memory=10)
    #suppress output so that it runs faster
    
    tp.subpx_bias(tp.locate(spheres[0], 31, invert=False, minmass=1000))
    fig00, ax00 = plt.subplots()

    #plot the trajectory of the sphere over the video
    pixtoum = 4.8 #pixel is 4.8 um for old high speed camera
    tp.plot_traj(t, ax=ax00, label=False, mpp = pixtoum)
    
    ax00.set_xlabel(r'x [$ \mu m$]')
    ax00.set_ylabel(r'y [$ \mu m$]')
    ax00.set_title("Spheres' Traces")



    ypx = t.loc[:,'y']
    xpx = t.loc[:,'x']
    spherenumber = t.loc[:,'particle']
    ypos = ypx * 4.8 * 10**(-6) #convert pixel to meter (pixel dimension 4.8x4.8um)
    xpos = xpx * 4.8 * 10**(-6) #convert pixel to meter

    totalspheres = max(t.loc[:,'particle']) + 1 
    xposlist = [[] for i in range(totalspheres)]
    yposlist = [[] for i in range(totalspheres)]


    #sort the dataframe to get the x,y position for each sphere
    for i in range(0, t.shape[0]):
        xposlist[spherenumber[i]].append(xpos[i])
        yposlist[spherenumber[i]].append(ypos[i])
        framenumlist[spherenumber[i]].append(framenum[i])
    
    nodrops = max(len(i) for i in xposlist)

    #sort the spheres by their position in the frame so it can be consistent across videos
    xposmeans = [np.average(xposlist[i]) for i in range(len(xposlist))]
    yposmeans = [np.average(yposlist[i]) for i in range(len(yposlist))]


    #make an array of the time for each frame in the video
    timeinc = 1/framerate 
    numframes = len(spheres) #gets number of frames in the video
    time = np.arange(0, numframes*timeinc, timeinc)
    freq = fft.rfftfreq(numframes, timeinc)
    segmentsize = round(framerate/4)


    xASDlist = [[] for i in range(totalspheres)]
    yASDlist = [[] for i in range(totalspheres)]

    spheredata = [[] for i in range(totalspheres)]
    sphere_pos_data = [[] for i in range(totalspheres)]


    Legendx = []
    Legendy = []
    figa, axa = plt.subplots()
    figb, axb = plt.subplots()

    fftbinning = 2048
    figs={}
    axs={}
    for i in range(0,len(xposlist)):
        
        if len(xposlist[i]) < nodrops:
            print('Sphere ' + str(i) + ' drops frames')
        else:
            frames = framenumlist[i]
            xcentered = xposlist[i] - np.mean(xposlist[i])
            xfreq, xPSD = welch(xcentered, framerate, 'hann', segmentsize, segmentsize/2, fftbinning, 'constant', True, 'density', 0,'mean')
            xASD = np.sqrt(xPSD)
            xASDlist[i] = np.vstack((xfreq,xASD)).T
            
            axa.semilogy(xfreq, xASD, linewidth=2)
            Legendx.append('Sphere ' + str(i))
    
            ycentered = yposlist[i] - np.mean(yposlist[i])
            yfreq, yPSD = welch(ycentered, framerate, 'hann', segmentsize, segmentsize/2, fftbinning, 'constant', True, 'density', 0,'mean')
            yASD = np.sqrt(yPSD)
            yASDlist[i] = np.vstack((yfreq,yASD)).T
            
            axb.semilogy(yfreq, yASD, linewidth=2)
            Legendy.append('Sphere ' + str(i))
            
            spheredata[i] = np.vstack((xcentered, ycentered)).T
            sphere_pos_data[i] = np.vstack((frames, xcentered, ycentered)).T


    axa.grid()
    axa.set_xlabel('Frequency [Hz]', fontsize=18)
    axa.set_ylabel(r'ASD [$m/ \sqrt{Hz}$]', fontsize=18)
    axa.legend(Legendx, fontsize=12, bbox_to_anchor=(1.04, 0), loc="lower left", borderaxespad=0)
    axa.set_title('X motion ASD', fontsize=22)
    axa.tick_params(axis='both', which='major', labelsize=12)
    #axa.set_xlim(8,167)
    for location in ['left', 'right', 'top', 'bottom']:
        axa.spines[location].set_linewidth(1)

    axb.grid()
    axb.set_xlabel('Frequency [Hz]', fontsize=18)
    axb.set_ylabel(r'ASD [$m/ \sqrt{Hz}$]', fontsize=18)
    axb.legend(Legendy, fontsize=12, bbox_to_anchor=(1.04, 0), loc="lower left", borderaxespad=0)
    axb.set_title('Y motion ASD', fontsize=22)
    axb.tick_params(axis='both', which='major', labelsize=12)
    #axb.set_xlim(8,167)
    for location in ['left', 'right', 'top', 'bottom']:
        axb.spines[location].set_linewidth(1)
    
    
    if saveposdata:
        savename = savename + '.h5'
        hf = h5py.File(savename, 'w')
        g1 = hf.create_group('position')
        g1.attrs.create('framerate (fps)', framerate)
        g2 = hf.create_group('X_psd')
        g2.attrs.create('framerate (fps)', framerate)
        g2.attrs.create('FFT bins', fftbinning)
        g2.attrs.create('FFT segment size', segmentsize)
        g3 = hf.create_group('Y_psd')
        g3.attrs.create('framerate (fps)', framerate)
        g3.attrs.create('FFT bins', fftbinning)
        g3.attrs.create('FFT segment size', segmentsize)
        for sphnum in range(len(spheredata)):
            d1 = g1.create_dataset('Spot ' + str(sphnum), data=sphere_pos_data[sphnum])
            d1.attrs.create('range (m)', [np.ptp(sphere_pos_data[sphnum][:,1]), np.ptp(sphere_pos_data[sphnum][:,2])])
            d1.attrs.create('rms (m)', [np.sqrt(np.mean((sphere_pos_data[sphnum][:,1])**2)), np.sqrt(np.mean((sphere_pos_data[sphnum][:,2])**2))])
            d1.attrs.create('Camera frame location (m)', [xmeanssorted[sphnum], ymeanssorted[sphnum]])
            g2.create_dataset('Spot ' + str(sphnum), data=xASDlist[sphnum])
            g3.create_dataset('Spot ' + str(sphnum), data=yASDlist[sphnum])
        hf.close()


    return totalspheres

#     rmsparsevalcheck0 = np.mean(x0centered**2)
#     psdparsevalcheck0 = 1/(numframes*timeinc) * np.sum(x0PSD)
#     print(rmsparsevalcheck0)
#     print(psdparsevalcheck0)
    
#     rmsparsevalcheck1 = np.mean(x1centered**2)
#     psdparsevalcheck1 = 1/(numframes*timeinc) * np.sum(x1PSD)
#     print(rmsparsevalcheck1)
#     print(psdparsevalcheck1)




In [ ]:
path = r"C:\Users\Ben\Documents\Image processing jupyter notebooks"
os.chdir(path)
filename = 'beforeAODjitter.avi'
framerate = 318.07
[spheres, f] = processmovie(filename, framerate)

PSDmaker(spheres, f)

In [ ]:
path = r"C:\Users\Ben\Documents\Image processing jupyter notebooks"
os.chdir(path)
filename = 'beam jitter pre chamber.avi'
framerate = 304.23
[spheres, f] = processmovie(filename, framerate)

PSDmaker(spheres, f)

deprecated pixel format used, make sure you did set range correctly


In [ ]:
t.loc[:,'particle']

In [ ]:
## now fit with Lorentzian

def lorentzian(f, f0, gam, cal_fac):
  kb = 1.38e-23 # Boltzmann's constant, SI units
  temp = 293 # Room temp, K
  m = 1e-12 # Mass given in problem set, kg
  omega = 2*np.pi*f
  omega0 = 2*np.pi*f0
  return 1/(cal_fac)**2 * 2*kb*temp/m * gam/((omega0**2 - omega**2)**2 + omega**2*gam**2)

In [ ]:
fig=plt.figure()
coords = ["Sphere 1 X", "Sphere 1 Y", "Sphere 2 X", "Sphere 2 Y"]
cal_facs = [] # array to store the calibration factors

NFFT = 2000

for idx, coord in enumerate(coords):

  dat, freqs = psd(xorig[:,idx], Fs=framerate, NFFT=NFFT)
  init_guess = [freqs[np.argmax(dat)],100,1e-7] # guess for the initial parameters
  best_params, cov = curve_fit(lorentzian, freqs, dat, p0=init_guess)

  plt.subplot(1,4,idx+1)
  plt.semilogy(freqs, dat, 'k', label="Data")
  plt.plot(freqs, lorentzian(freqs,*best_params), 'r', lw=1, label="Fit")
  plt.xlabel('Frequency [Hz]')
  if(idx == 0):
    plt.ylabel("PSD [m$^2$/Hz]")
    plt.legend()
  plt.title(coord)

  cal_facs.append(best_params[2]) ## save the calibration factor for later use

fig.set_size_inches(20,5)
cal_facs